## Webscraping Bundesliga

The dataset from Kaggle had wrong match data for the seasons 2015/2016 to 2019/2020. This script scrapes the correct data for these seasons and exports them as an XLSX file. The new data will then replace the wrong data in the Excel source file for Tableau.

In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [20]:
# Initialize lists to store the extracted data
season = []
matchday_nr = []
home_teams = []
away_teams = []
goals_home = []
goals_away = []

In [21]:
target_seasons = [2016, 2017, 2018, 2019, 2020]
target_matchdays = list(range(1, 35))

In [22]:
# URL to scrape
url_template = "https://www.fussballdaten.de/bundesliga/"

# Loop throgh all target seasons and matchdays
for season_ in target_seasons:
    for matchday_ in target_matchdays:
        url = url_template + str(season_) + "/" + str(matchday_) + "/"
        
        # Send a GET request to the URL
        response = requests.get(url)

        # Create a BeautifulSoup object from the response content
        soup = BeautifulSoup(response.content, "html.parser")

        # Find all elements with class="ergebnis"
        ergebnis_elements = soup.find_all(class_="ergebnis")

        # Extract the required information from each element
        for element in ergebnis_elements:
            # Extract HOME
            title = element["title"]
            home_team = title.split("Spieldetails:")[1].split("gegen")[0].strip()
            home_teams.append(home_team)

            # Extract AWAY
            away_team = title.split("gegen")[1].split("(")[0].strip()
            away_teams.append(away_team)

            # Extract GOALS_HOME and GOALS_AWAY
            goals = element.find("span").get_text().split(":")
            goals_home.append(int(goals[0]))
            goals_away.append(int(goals[1]))
            
            # Add season and matchday info from iterations
            season.append(int(season_))
            matchday_nr.append(int(matchday_))

In [23]:
# Create a pandas DataFrame
data = {
    "SEASON": season,
    "MATCHDAY_NR": matchday_nr,
    "HOME": home_teams,
    "AWAY": away_teams,
    "GOALS_HOME": goals_home,
    "GOALS_AWAY": goals_away
}
df = pd.DataFrame(data)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1836 entries, 0 to 1835
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   SEASON       1836 non-null   int64 
 1   MATCHDAY_NR  1836 non-null   int64 
 2   HOME         1836 non-null   object
 3   AWAY         1836 non-null   object
 4   GOALS_HOME   1836 non-null   int64 
 5   GOALS_AWAY   1836 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 86.2+ KB


In [27]:
# Export as xlsx without index column
df.to_excel("Correction.xlsx", index=False)